# Notebook for automated multi-year data visualization from copernicus marine server

**USER INPUT**

In [1]:
# List with start and end dates organized as tuples
dates = [("2008-05-01T00:00:00", "2008-10-31T00:00:00"), 
         ("2024-05-01T00:00:00", "2024-10-31T00:00:00")]

###------------------------------------------------------###

# metadata for plotting
dataset_id = "SST_MED_SST_L3S_NRT_OBSERVATIONS_010_012_b"

variables = ["adjusted_sea_surface_temperature"]

DOI = "https://doi.org/10.48670/moi-00171"

spatial_resolution = "0.01° × 0.01°"

temporal_resolution = "Daily"

**Dependencies**

In [2]:
# necessary libraries to run this NOTEBOOK
# if not already installed, install via conda-forge channel using Ana(Mini)conda, Micromamba or pip
import copernicusmarine
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from esda.getisord import G_Local
from libpysal.weights import Queen
from pyproj import CRS
import contextily as ctx
import rioxarray as rio
import rasterio
import pathlib
from shapely.geometry import Point
import folium
import os

In [3]:
# retrieving realtive paths
NOTEBOOK_DIRECTORY = pathlib.Path().resolve()
OUTPUT_DIRECTORY = NOTEBOOK_DIRECTORY / "output" / "multi_year" / "plots"
DATA_DIRECTORY = NOTEBOOK_DIRECTORY / "output" / "multi_year" / "data"

os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)
os.makedirs(DATA_DIRECTORY, exist_ok=True)

## Functions

In [4]:
# function for fetching copernicus dataset from the server 
def get_cm_dataset(start, end, dataset_id, variables):
    return copernicusmarine.open_dataset(
        dataset_id=dataset_id, # default: product ID for sea surface temperature, change if desired
        variables=variables, # default: seasurface temperature
        minimum_longitude=19.22659983450641, # default: bounding box for the whole agean, change if desired 
        maximum_longitude=28.439441984120553,
        minimum_latitude=34.62160284496615,
        maximum_latitude=40.9634662781889,
        start_datetime=start,
        end_datetime=end,
    )

In [5]:
# function for pre-processing fetched dataset
def sst_pre_processing(dataset):
    # renaming data variable
    dataset = dataset.rename({"adjusted_sea_surface_temperature": "sst"})
    # converting Kelvin to Celsius
    dataset["sst"] = dataset["sst"]-273.15

    return dataset

In [6]:
# function for calculating basic stats on sst
def sst_bstats(dataset_preprocessed):
    # computing mean
    sst_mean = dataset_preprocessed["sst"].mean(dim="time", skipna=True) # collapsing the 3D dataset to 2D by calculating the mean over time 
    # computing median
    sst_median = dataset_preprocessed["sst"].median(dim="time", skipna=True)

    return sst_mean, sst_median

## Import

In [7]:
# Dictionary to store datasets by year
datasets = {}

# Loop over dates, extracting the year dynamically
for start, end in dates:
    year = start[:4]  # Extract the year from the start date
    datasets[int(year)] = get_cm_dataset(start, end, dataset_id, variables)  # Convert year to int

INFO - 2025-03-10T16:02:04Z - Selected dataset version: "202311"
INFO - 2025-03-10T16:02:04Z - Selected dataset part: "default"
INFO - 2025-03-10T16:02:10Z - Selected dataset version: "202311"
INFO - 2025-03-10T16:02:10Z - Selected dataset part: "default"


## Processing

In [8]:
# Update datasets with preprocessed values by calling function
datasets = {year: sst_pre_processing(ds) for year, ds in datasets.items()} # dictionary comprehension, pre_processing dictionary while looping

In [9]:
# Dictionary to store computed SST statistics
sst_stats = {}

# Loop over datasets, compute statistics, and store in new dictionary
for year, dataset in datasets.items():
    sst_mean, sst_median = sst_bstats(dataset)  # Apply function
    sst_stats[year] = {"mean": sst_mean, "median": sst_median}  # Store results (Datasets are now collapsed to 2D and now stored in form of a DataArray (not Dataset anymore))

In [12]:
# Dictionary to store GeoDataFrames
sst_stats_gdf = {}

# Loop over computed SST stats
for year, stats in sst_stats.items(): # year = key of dictionary, stats= data/values stored accessible with key 
    # Convert mean SST to DataFrame and reset index
    df_mean = stats["mean"].to_dataframe(name="Mean SST").reset_index()
    df_median = stats["median"].to_dataframe(name="Median SST").reset_index()
    
    # Merge mean and median DataFrames on latitude & longitude
    df = df_mean.join(df_median.set_index(["latitude", "longitude"]), on=["latitude", "longitude"])
    
    # Drop NaNs
    df = df.dropna()

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df,
        geometry=[Point(xy) for xy in zip(df["longitude"], df["latitude"])],  # Create geometry column
        crs="EPSG:4326"  # Set coordinate reference system (WGS 84)
    )

    # save GeoDataFrame to file
    gdf.to_file(DATA_DIRECTORY / f"{year}.gpkg")

    # Add a unique Geo-ID for Folium (based on index)
    gdf["geoid"] = gdf.index.astype(str)

    # Store in dictionary
    sst_stats_gdf[year] = gdf

## Visualization

In [11]:
# Loop through the years and generate separate static maps for Mean & Median SST
for year, gdf in sst_stats_gdf.items():
    # Reproject to Web Mercator (EPSG:3857) for compatibility with basemap
    gdf = gdf.to_crs(epsg=3857)

    # Define metadata for attribution
    attribution = (f"Author: Moritz Mühlbauer 2025 in cooperation with Archipelagos\n"
                   f"Source: Generated using E.U. Copernicus Marine Service Information,\n"
                   f"DOI: {DOI}\n"
                   f"Dataset ID: {dataset_id}\n"
                   f"Spatial resolution: {spatial_resolution}\n"
                   f"Temporal resolution: {temporal_resolution}\n"
                   f"Projection: WEB MERCATOR (EPSG:3857)\n"
                   f"Basemap: Esri World Imagery")

    # --- Mean SST Map ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot points with very small marker size
    sc = gdf.plot(
        ax=ax, 
        column="Mean SST", 
        cmap="RdBu_r",  
        markersize=1,  
        legend=True,
        alpha=0.8
    )

    # Add attribution text
    ax.text(0, -0.26, attribution, fontsize=5.5, transform=ax.transAxes, 
            fontweight='normal', bbox={'facecolor': 'none', 'pad': 0.01, 'edgecolor': 'none'})

    # basemap, check the offering by running ctx.providers.keys()
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, attribution="")  

    ax.set_title(f"Mean Sea Surface Temperature May - October / ({year})", fontsize=14)
    ax.set_xlabel("Easting (meters)")
    ax.set_ylabel("Norhting (meters)")

    # Adjust colorbar label
    cbar = sc.get_figure().get_axes()[1]  # Get colorbar
    cbar.set_ylabel("C°")  # Add temperature unit

    # Save Mean SST Map with DPI 600
    mean_output_file = os.path.join(OUTPUT_DIRECTORY, f"sst_mean_{year}.png")
    plt.savefig(mean_output_file, dpi=600, bbox_inches="tight")  
    plt.close(fig)  # Close the figure to free memory
    print(f"Mean SST map for {year} successfully saved to {mean_output_file}")

    # --- Median SST Map ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot points with very small marker size
    sc = gdf.plot(
        ax=ax, 
        column="Median SST", 
        cmap="RdBu_r",  
        markersize=1,  
        legend=True,
        alpha=0.8
    )

    # Add attribution text
    ax.text(0, -0.26, attribution, fontsize=5.5, transform=ax.transAxes, 
            fontweight='normal', bbox={'facecolor': 'none', 'pad': 0.01, 'edgecolor': 'none'})

    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, attribution="")  

    ax.set_title(f"Median Sea Surface Temperature May- October ({year})", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    # Adjust colorbar label
    cbar = sc.get_figure().get_axes()[1]  # Get colorbar
    cbar.set_ylabel("C°")  # Add temperature unit

    # Save Median SST Map with DPI 600
    median_output_file = os.path.join(OUTPUT_DIRECTORY, f"sst_median_{year}.png")
    plt.savefig(median_output_file, dpi=600, bbox_inches="tight")  
    plt.close(fig)  # Close the figure to free memory
    print(f"Median SST map for {year} successfully saved to {median_output_file}")


Mean SST map for 2008 successfully saved to /Users/moritzmuhlbauer/Copernicus/output/multi_year/plots/sst_mean_2008.png
Median SST map for 2008 successfully saved to /Users/moritzmuhlbauer/Copernicus/output/multi_year/plots/sst_median_2008.png
Mean SST map for 2024 successfully saved to /Users/moritzmuhlbauer/Copernicus/output/multi_year/plots/sst_mean_2024.png
Median SST map for 2024 successfully saved to /Users/moritzmuhlbauer/Copernicus/output/multi_year/plots/sst_median_2024.png
